In [241]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

from matplotlib import rc
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import itertools

In [242]:
data=pd.read_csv('./zillow_data.csv')

In [243]:
data

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [244]:
#pd.get_option("display.max_rows")
pd.set_option("display.max_rows", 500)

In [245]:
#separate out areas within the state of California
data_ca = data.loc[data.State=='CA']
data_ca.Metro.unique()

array(['San Francisco', 'Los Angeles-Long Beach-Anaheim', 'Sacramento',
       'Napa', 'San Diego', 'Fresno', 'Vallejo', 'Riverside', 'Ventura',
       'San Jose', 'Bakersfield', 'Visalia', 'Hanford', 'Santa Cruz',
       'Modesto', 'Redding', 'Santa Maria-Santa Barbara', 'Stockton',
       'Salinas', 'San Luis Obispo', 'Santa Rosa', 'Chico', 'El Centro',
       'Yuba City', 'Merced', 'Ukiah', 'Truckee', 'Madera', 'Eureka',
       'Susanville', nan, 'Clearlake'], dtype=object)

In [246]:
#separate out the Metro areas of San Francisco 
data_sf = data[(data.Metro == 'San Francisco') & (data.State == 'CA')]
print(data_sf.City.unique())
print(data_sf.shape)
data_sf.head()

['San Francisco' 'Pittsburg' 'Alameda' 'Fremont' 'Hayward' 'Antioch'
 'Brentwood' 'Union City' 'Dublin' 'South San Francisco' 'Martinez'
 'Daly City' 'Oakland' 'Livermore' 'San Leandro' 'Castro Valley'
 'San Mateo' 'Burlingame' 'San Bruno' 'Pleasanton' 'Concord' 'Menlo Park'
 'San Rafael' 'Foster City' 'Richmond' 'Pacifica' 'Pleasant Hill'
 'San Ramon' 'Newark' 'Redwood City' 'Oakley' 'Danville' 'Mill Valley'
 'San Carlos' 'Belmont' 'El Cerrito' 'Lafayette' 'Walnut Creek' 'Berkeley'
 'Novato' 'San Lorenzo' 'El Sobrante' 'Millbrae' 'Hercules' 'Albany'
 'Pinole' 'Orinda' 'Half Moon Bay' 'San Anselmo' 'Tiburon' 'Discovery Bay'
 'Sausalito' 'Alamo' 'Moraga' 'Kentfield' 'Clayton' 'Corte Madera'
 'Fairfax' 'Larkspur' 'Rodeo' 'Portola Valley' 'Atherton' 'Brisbane'
 'Crockett' 'Montara' 'Moss Beach' 'Ross' 'La Honda' 'Woodacre'
 'Stinson Beach']
(134, 272)


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500
27,97771,94565,Pittsburg,CA,San Francisco,Contra Costa,28,139200.0,138300.0,137500.0,...,394900,398400,401600,405400,408600,410900,413700,417900,424300,430100
64,97711,94501,Alameda,CA,San Francisco,Alameda,65,222400.0,222300.0,222400.0,...,965100,975000,987800,1000000,1009900,1021000,1032400,1042000,1053000,1059700
76,97565,94110,San Francisco,CA,San Francisco,San Francisco,77,213900.0,214800.0,216000.0,...,1503000,1506400,1513500,1525200,1533600,1536500,1536200,1544400,1570000,1594000
186,97743,94536,Fremont,CA,San Francisco,Alameda,187,223200.0,223200.0,223300.0,...,960100,968200,978000,989600,1002400,1018700,1033100,1045400,1061200,1072700


In [247]:
#separate out the Metro areas of San Jose
data_sj = data[(data.Metro == 'San Jose') & (data.State == 'CA')]
print(data_sj.City.unique())
print(data_sj.shape)

['San Jose' 'Santa Clara' 'Milpitas' 'Cupertino' 'Sunnyvale' 'Campbell'
 'Gilroy' 'Morgan Hill' 'Hollister' 'Mountain View' 'Palo Alto' 'Saratoga'
 'Los Gatos' 'Los Altos' 'Stanford' 'Lexington Hills' 'San Martin'
 'San Juan Bautista' 'Aromas']
(55, 272)
